## 1-1. 들어가며

여러분 안녕하세요.

지금부터 쏘카 비즈니스데이터 팀에서 고민하고 있는 **수요 예측 문제** 에 대해 공유하고, 해결해보는 과정을 함께 학습하겠습니다. 여러분이 쏘카의 데이터 사이언티스트라고 상상하면서 문제해결 과정을 수행한다면 더욱더 재밌게 본 과정을 수행할 수 있을 것입니다.

쏘카는 차가 필요한 순간, 사람들에게 카셰어링을 서비스하는 기업입니다.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/ssokaimiji.max-800x600.png)

카셰어링 산업에서 수요 예측은 매우 중요한 의미를 가집니다. 정확한 수요 예측으로부터 가격, 인벤토리, 배차 등이 결정되기 때문입니다. 쏘카에서 차량에 대한 수요 예측을 왜 중요한 이슈로 다루고 있는지 느낌이 오시나요?

하지만 바로 수요 예측 분석을 수행하는 것은 매우 위험합니다. 정확하게 환경과 문제를 인지하고, 그에 맞는 수집해야 할 데이터와 보유하고 있는 데이터를 확인하고 살펴본 후에 수요 예측 분석을 수행해야 비즈니스적으로 의미 있는 분석 결과를 도출할 수 있기 때문입니다.

의미 있는 수요 예측 분석을 위한 일련의 과정을 지금부터 함께 학습해봅시다.

### 학습 목표

___

-   쏘카에서 차량 수요 예측이 가지는 중요성을 이해한다.
-   EDA(탐색적 데이터 분석)의 중요성을 파악하고, 데이터를 분석한다.
-   상관관계 분석과 선형 분석을 이해하고, 수행한다.

### 목차

___

1.  들어가며
2.  차량 수요 예측을 위한 환경 인식과 문제 정의
3.  데이터 및 라이브러리 준비
4.  EDA
5.  상관관계 분석
6.  선형 회귀 분석
7.  마무리하며

### 준비물

___

클라우드 환경에서는 Cloud shell에서 아래 명령어를 입력하여 준비된 데이터를 연결시켜 주세요.

```
$ mkdir -p ~/aiffel/demand_forecasting/data
$ ln -s ~/data/* ~/aiffel/demand_forecasting/data
```

## 1-2. 차량 수요 예측을 위한 환경 인식과 문제 정의

쏘카에서 고민하고 있는 수요 예측을 수행하기 위해 영상을 통해 수요와 공급의 의미, 카셰어링 산업에 어떻게 적용되는지, 자주 사용할 용어 정리를 하겠습니다.

<iframe src="https://player.vimeo.com/video/605482321?badge=0&amp;autopause=0&amp;player_id=0&amp;app_id=58479&amp;h=9b2db6d858" width="740" height="480" frameborder="0" allow="autoplay; fullscreen; picture-in-picture" allowfullscreen="" title="차량 수요 예측을 위한 환경 인식과 문제 정의"></iframe>

어떤 데이터를 사용해서 어떻게 수요 예측을 진행해야 할지 정리되셨나요? 영상의 내용을 퀴즈를 통해 정리해 보도록 합시다.

Q1. 차량 수요 예측 모델링의 예측 대상과 features는 무엇인가요?

예시답안

예측대상은 target date의 가동률이고, features는 target date의 D-1~D-7 사전 가동률, target date의 D-1~D-7 존 클릭 수, target date의 일기예보이다.

Q2. 차량 수요 예측을 하려면 어떤 방법을 사용하면 좋을지 토론해 보세요.

예시답안

feature간 상관관계를 분석한다, 시계열 분석을 해본다 등

## 1-3. 데이터 및 라이브러리 준비

### 라이브러리 불러오기

___

이제 필요한 라이브러리를 불러와 볼까요?

In [ ]:
import os

# 데이터 처리
import pandas as pd
import numpy as np

# 시각화
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import seaborn as sns

# 분석
import statsmodels.api as sm
import scipy

# 그래프 글꼴 설정
plt.rc('axes', unicode_minus=False)
plt.rc('font', family='NanumBarunGothic')

print("슝~")

### 데이터 접근하기 - 데이터 기본 파악

___

차량 수요 예측 모델링을 위해 사전 가동률 데이터, 존 클릭수 데이터, 날씨 데이터를 불러옵시다.

사전 가동률이란 당일(D-day) 전에 미리 예약이 발생하여 가동률이 잡힌 것을 의미해요. 내일 이용할 것을 당일이 아닌 오늘, 혹은 그 전에 미리 예약하는 유저들도 있겠죠. 예를 들면 2021년 9월 3일 금요일 현재, 2021년 9월 4일 토요일을 이용 시작일로 하는 카셰어링 예약이 있을 때, D-1일 사전가동률이 잡히게 돼요. 존 클릭 수는 특정 존(쏘카가 비치된 주차장)을 클릭한 수입니다.

In [ ]:
file_path_pre = os.getenv('HOME') + '/aiffel/demand_forecasting/data/pre_operation.csv'
file_path_click = os.getenv('HOME') + '/aiffel/demand_forecasting/data/click_data.csv'
file_path_weather = os.getenv('HOME') + '/aiffel/demand_forecasting/data/weather.csv'

df_pre = pd.read_csv(file_path_pre)
df_click = pd.read_csv(file_path_click)
df_weather = pd.read_csv(file_path_weather)

print('슝~')

데이터가 어떻게 구성되었는지 확인해 볼까요?

영상에서 언급되었던 용어가 보일 거예요. 다음 단계로 넘어가기 전에 각 데이터의 컬럼의 의미를 확인해 봅시다.

In [ ]:
df_pre.head()

-   target\_dates: 고객이 쏘카 APP에 접속하여 start\_date로 삼은 날짜
-   op\_rate\_0d\_all\_cars: target\_date 모든 차량의 d-day 가동률. 간단히 '최종 가동률'이라고 하겠습니다.
-   op\_rate\_1d\_all\_cars: target\_date 모든 차량의 d-1일 전 사전 가동률
-   op\_rate\_0d\_major\_cars: target\_date 주력 차량의 d-day 가동률( 경형, 준중형, 소형SUV )
-   op\_rate\_1d\_major\_cars: target\_date 주력 차량의 d-1일 전 사전 가동률( 경형, 준중형, 소형SUV )

In [ ]:
df_click.head(15)

-   click\_d\_day: target\_date를 start\_date로 삼는 D-day 당일의 APP접속량. 간단히 'D-day의 존 클릭 수'라고 부를 것입니다.
-   click\_d\_1: target\_date를 start\_date로 삼는 D-1일의 APP접속량. 간단히 '1일 전 존 클릭 수'라고 부를 것입니다.

In [ ]:
df_weather.head()

-   is\_clean: 강우 여부(강수/강우가 측정되면 0, 측정되지 않으면 1)
-   avg\_precipitation: 해당 날짜의 평균 강수량
-   avg\_temperature: 해당 날짜의 평균 기온

## 1-4. EDA (1) : 데이터 일관성 확인하기

EDA(Exploratory Data Analysis, 탐색적 데이터 분석)는 변수들의 분포를 도식화 및 도표화하여 대응 관계를 파악하고, 요약적 통계량을 생성하는 등 기술 통계량을 중심으로 데이터에서 정보를 찾으려는 노력이에요. 간단한 방법으로 데이터 전체를 분석하고, 그 패턴을 찾을 수 있게 하는 매우 중요한 분석 과정이죠. 이러한 분석 과정에 앞서 필요한 과정이 바로 데이터의 일관성(Consistency)을 확인하는 단계입니다. 데이터의 일관성을 검증하지 않고, 무턱대고 EDA를 수행한다면 Raw data로부터 의미 있는 정보, 올바른 정보를 도출할 수 없어요!

이제부터 데이터의 일관성 확인을 위해 데이터에 결측치가 있는지와 데이터 타입을 알아봅시다.

In [ ]:
df_pre.info()

사전 가동률 데이터는 총 17개의 컬럼과 939일간의 데이터가 있고, 데이터타입은 target\_dates를 제외하고는 모두`float64`에요.

In [ ]:
df_click.info()

존 클릭 수 데이터는 총 9의 컬럼과 793일간의 데이터가 있고, 데이터 타입은 target\_dates를 제외하고 모두`int64`입니다.

df_weather.info()

마지막으로 날씨 데이터는 총 4개의 컬럼이 있고, 총 1,238일간의 데이터로 이루어져 있습니다. 데이터타입은 `int64`, `float64`이네요. 또한 날씨 데이터의 `avg_temperature`의 데이터 개수가 다른 컬럼의 데이터보다 19개 적은 것을 알 수 있어요.

이제 우선 데이터타입을 통일시키겠습니다.

먼저 날짜 컬럼의 데이터를 `object`에서 `datetime`으로 변경할게요.

In [ ]:
df_pre['target_dates'] = pd.to_datetime(df_pre['target_dates'], format='%Y-%m-%d')
df_click['target_dates'] = pd.to_datetime(df_click['target_dates'], format='%Y-%m-%d')
df_weather['date'] = pd.to_datetime(df_weather['date'], format='%Y-%m-%d')

print("슝~")

변경 결과를 확인해 볼까요?

df_pre.info()

In [ ]:
df_click.info()

In [ ]:
df_weather.info()

모두 잘 변경되었군요. 이제 결측치를 확인해 봅시다.

### 결측치 처리

___

결측치를 처리하는 방법에는 여러 가지가 있어요.

1.  결측치 row를 없애는 방법
2.  해당 column의 대표값( 평균값, 중간값, 최빈값 등)을 넣는 방법
3.  직전 row/직후 row 값을 넣는 방법
4.  직전 row와 직후 row의 중간값(선형 보간)을 넣는 방법

이렇게 4개의 방법이 있고, 상황에 맞는 방법을 선택해서 결측치를 처리하면 돼요.

**1\. 사전 가동률 데이터**

첫 번째로 사전 가동률 데이터의 결측치 여부를 `difference()` 함수를 사용해 살펴볼게요.

In [ ]:
set(pd.date_range(min(df_pre.target_dates), max(df_pre.target_dates))).difference(set(df_pre.target_dates))

사전 가동률 데이터에는 결측치가 존재하지 않아요.

**2\. 존 클릭 수 데이터**

In [ ]:
set(pd.date_range(min(df_click.target_dates), max(df_click.target_dates))).difference(set(df_click.target_dates))

최초 날짜인 `2019-01-23` 이후부터 `2019-08-30`의 행이 거의 존재하지 않음을 확인할 수 있어요. 좀 더 자세히 확인해 볼까요?

In [ ]:
print(df_click[df_click.target_dates<'2019-08-30'])

두 날짜 사이의 클릭 정보가 모두 0인 것으로 보아 해당 행들이 잘못 들어가 있다는 것을 알 수 있죠. 그러니 존 클릭수 데이터의 결측치는 제거해도 괜찮아요.

In [ ]:
df_click.drop(df_click[df_click.target_dates<'2019-08-30'].index, inplace=True)

print("삭제한 결과를 확인해 볼게요")
set(pd.date_range(min(df_click.target_dates), max(df_click.target_dates))).difference(set(df_click.target_dates))

**3\. 날씨 데이터**

마지막으로 날씨 데이터의 결측치를 확인해 보죠.

In [ ]:
set(pd.date_range(min(df_weather.date), max(df_weather.date))).difference(set(df_weather.date))

날씨 데이터는 일부 날짜가 중간중간 존재하지 않아요. 이럴 경우에는 결측치의 양이 많지 않기 때문에 기상청 자료를 직접 확인해 채워 넣는 것도 한 가지 방법이에요. 만약 결측치의 양이 많다면 API를 이용해 자료를 가져와야겠죠?

하지만 평균 기온은 채워 넣을 수 있는 다른 방법이 있으니 기상청 자료를 확인하는 것은 생략할게요. 먼저 존재하지 않는 날짜를 만들고, 인덱스를 리셋시킬게요.

In [ ]:
df_weather = df_weather.append(
    pd.DataFrame({
        'date': pd.date_range(min(df_weather.date), max(df_weather.date)).difference(df_weather.date),
        'is_clean': [1]*6,
        'avg_precipitation': [0]*6
    })
)

print("슝~")

날씨 데이터의 결측치를 다시 확인해 볼게요.

In [ ]:
# 날씨 데이터에서의 결측치 확인
df_weather[df_weather['avg_temperature'].isnull()]

결측치 중에서 2021년 3월의 결측치가 가장 많은 것을 알 수 있어요. 결측치를 포함한 2021년 3월의 데이터만 살펴볼게요.

In [ ]:
# 2021년 03월만 추려서 봐보면,
df_weather['month'] = df_weather['date'].dt.month
df_weather['year'] = df_weather['date'].dt.year
df_weather[(df_weather['month']==3) & (df_weather['year']==2021)]

Q3. 날씨 데이터의 결측치를 어떻게 처리하면 좋을까요?

예시답안

직전과 직후 row의 중간값(선형 보간)을 넣는다.

결측치를 처리하는 각 방법의 장단점은 아래와 같아요.

**1\. 결측치 row를 없애는 방법**

우리가 가지고 있는 데이터의 총수는 사전가동률 데이터 939개, 존 클릭 수 데이터 793개, 날씨 데이터 1,213개로, 분석하기에는 데이터 규모가 매우 작은 편이에요. 따라서 결측치를 없애는 것은 좋은 방법이 아니에요. 또한 시계열 자료에서 결측치가 존재하는 경우, 기존에 사용하는 방법(ARIMA)을 그대로 적용하기 어렵고 모델링 난도도 크게 높아져요. 그러나 만약 데이터가 시계열 자료가 아니거나 제거 후의 데이터 규모가 충분히 크다고 판단되면 결측치 row를 제거하는 것은 좋은 방법이에요.

**2\. 해당 column의 대표값( 평균값, 중간값, 최빈값 etc )을 넣는 방법**

대표값은 말 그대로 대표값일 뿐, 정말 해당 컬럼의 내용을 충분히 대표할 수 있을지 판단해 봐야 해요. 만약 날씨 데이터의 평균 기온 결측값을 대표값으로 넣을 경우, 12월 30일의 전국 평균 기온의 결측값이 연평균 기온 값이 되므로 데이터 왜곡을 초래할 수 있어요. 이를 보완하기 위해 해당 월평균, 해당 주 대표값으로 결측치를 대신할 수 있어요.

**3\. 직전 row/직후 row 값을 넣는 방법**

Seasonality를 갖는 데이터의 경우, 직전 혹은 직후의 row 값을 넣는 것은 대표값보다 더 나은 방법이 될 수 있어요. 해당 시기의 seasonality를 갖고 있기에 이질감이 매우 적기 때문이죠. 하지만 그래프를 그렸을 때 매끄럽지 않다는 단점이 있어요.

**4\. 직전 row와 직후 row의 중간값(선형 보간)을 넣는 방법**

날씨 데이터의 평균 기온 데이터의 경우, 선형 보간을 사용하는 것이 가장 이상적인 방법 중 하나에요. 기온이 어제와 오늘 갑자기 급변하는 경우는 극히 드물 뿐더러, 어제의 기온과 내일의 기온을 매끄럽게 연결해줄 수 있기 때문이죠. 또한 연속으로 결측값이 있어도 매끄럽게 결측치를 대신할 수 있어요. 따라서 여기에서는 선형 보간을 활용하여 결측치를 대신하기로 해요.

**선형 보간법**  
선형 보간법은 '1차원 직선상에서 끝점의 값이 주어졌을 때, 그 사이의 값을 추정하기 위해 직선거리에 따라 선형적으로 계산하는 방법'이에요. 정의만 보면 이해가 쉽지 않죠? 다시 말하면 두 점 사이의 값을 비례식을 통해 추정하는 건데요, 여기서는 간단하게 결측치 직전과 직후 값의 중간값을 계산함으로써 결측치를 처리하는 거죠.

선형 보간법에 대한 자세한 내용을 알고 싶다면 아래의 링크를 참고하세요.

-   [\[Algorithm\] 선형 보간법(Linear interpolation)](https://spiralmoon.tistory.com/entry/Algorithm-%EC%84%A0%ED%98%95-%EB%B3%B4%EA%B0%84%EB%B2%95-Linear-interpolation)

In [ ]:
# 선형 보간법 적용
df_weather.interpolate(method='linear', inplace=True)
df_weather['avg_temperature'] = df_weather['avg_temperature'].apply(lambda x: f'{x: .1f}') # avg_temperature의 타입을 변경합니다

print("슝~")

In [ ]:
# 적용하고 다시 2021년 03월의 데이터를 봐보자
df_weather[(df_weather['month']==3) & (df_weather['year']==2021)]

어떤가요? 선형 보간법을 적용하니 날씨 데이터가 그럴 듯해 보이죠?

In [ ]:
# df 정보 확인
df_weather.info()

하지만 평균 기온의 데이터타입이 `object`네요. 분석을 용이하게 하기 위해 데이터 타입을 `float64`로 바꿔 줄게요.

In [ ]:
# 평균 기온의 데이터타입을 float64로 바꿔준다
df_weather['avg_temperature'] = df_weather['avg_temperature'].astype(float)

df_weather.info()

자, 이제 데이터타입의 변경과 결측치 처리가 끝났으니 본격적으로 데이터를 살펴볼까요?

## 1-5. EDA (2) : 데이터 기본 속성 살펴보기

이제부터 전처리된 데이터를 활용하여 데이터의 분포를 살펴보려고 해요. `describe()`를 통해 통계량을 살펴보고, 시각화해 볼거에요.

### 존 클릭 수 데이터의 기본 분포 살펴보기

df_click.describe()

히스토그램을 통해 존 클릭 수 데이터의 분포를 살펴볼게요.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 8))

sns.kdeplot(df_click['click_d_day'])
sns.kdeplot(df_click[ 'click_d_1' ])
sns.kdeplot(df_click[ 'click_d_3' ])
sns.kdeplot(df_click[ 'click_d_5' ])

plt.legend(['d-day', 'd-1', 'd-3', 'd-5'])
plt.xlabel('클릭 수')
plt.ylabel('분포')
plt.title('d-day에 따른 클릭 수 분포', fontsize=15)
plt.show()

위의 히스토그램을 봤을 때, **D-day의 click 수 분포가 가장 높은 구간에 위치**함을 알 수 있어요. (약 20만 개의 클릭) 이를 통해 회원의 대부분은 미리 예약하기 보다는, **이용 당일에 예약하는 패턴**을 보임을 알 수 있어요.

### 날씨 데이터의 기본 분포 살펴보기

df_weather.describe()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 8))

sns.lineplot(
    data=df_weather,
    x='month',
    y='avg_precipitation',
    label='평균 강수량',
)

sns.lineplot(
    data=df_weather,
    x='month',
    y='avg_temperature',
    label='평균 기온'
)

plt.title('월간 날씨 추이', fontsize=15)
plt.xlabel('월(month)')
plt.ylabel('')
plt.show()

예상대로 여름에 평균 강수량과 평균 기온이 높군요.

### 사전 가동률 데이터의 기본 분포 살펴보기

In [ ]:
df_pre.describe()

사전 가동률 데이터는 컬럼이 많아서인지 한눈에 잘 들어오지 않네요. 판다스의 `melt()`를 사용해서 `op_rate_od_all_cars`부터 `op_rate_7d_all_cars`의 여러 컬럼을 `op_value`라는 하나의 컬럼으로 합쳐 봅시다. D-day부터 D-7까지의 날짜는 기준이 되는 컬럼인 `d-day`를 만들어서 확인할 수 있어요.

In [ ]:
df_pre_v1 = df_pre[['target_dates', 'op_rate_0d_all_cars', 'op_rate_1d_all_cars', 'op_rate_2d_all_cars', 'op_rate_3d_all_cars', 'op_rate_4d_all_cars', 'op_rate_5d_all_cars', 'op_rate_6d_all_cars', 'op_rate_7d_all_cars']]

df_pre_v1 = df_pre_v1.melt(
    id_vars='target_dates',
    value_vars=['op_rate_0d_all_cars', 'op_rate_1d_all_cars', 'op_rate_2d_all_cars', 'op_rate_3d_all_cars', 'op_rate_4d_all_cars', 'op_rate_5d_all_cars', 'op_rate_6d_all_cars', 'op_rate_7d_all_cars'],
    var_name='d-day',
    value_name='op_value',
)\
.replace('op_rate_0d_all_cars', 0)\
.replace('op_rate_1d_all_cars', 1)\
.replace('op_rate_2d_all_cars', 2)\
.replace('op_rate_3d_all_cars', 3)\
.replace('op_rate_4d_all_cars', 4)\
.replace('op_rate_5d_all_cars', 5)\
.replace('op_rate_6d_all_cars', 6)\
.replace('op_rate_7d_all_cars', 7)

df_pre_v1.head()

In [ ]:
df_pre_v1[df_pre_v1['target_dates']=='2019-11-20']

`target_dates`의 d-day를 0부터 7까지의 숫자로 볼 수 있으니 이전보다 훨씬 쉽게 데이터를 파악할 수 있지 않나요? 이제 lineplot을 통해 데이터의 분포를 확인해 봅시다.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 8))

sns.lineplot(
    data=df_pre_v1,
    x='d-day',
    y='op_value'
)

plt.show()

그래프가 d-day로 갈수록 사전 가동률이 높아지네요. d-day의 가동률이 가장 높은 것도 볼 수 있어요.

이제 데이터를 정렬하고 데이터 정보를 다시 확인해 봅시다.

In [ ]:
df_pre_v1 = df_pre_v1.sort_values(by='target_dates', ascending=True, ignore_index=True)
df_pre_v1.head()

df_pre_v1.info()

가동률을 시기별로 보기 위해서 `weeknum, year, yyyyww` 컬럼을 추가해 줄게요.

In [ ]:
df_pre_v1['weeknum'] = df_pre_v1['target_dates'].dt.isocalendar().week
df_pre_v1['year'] = df_pre_v1['target_dates'].dt.isocalendar().year
df_pre_v1['yyyyww'] = df_pre_v1['year'].astype(str) + '-' + df_pre_v1['weeknum'].astype(str)
df_pre_v1.head()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(11, 8))
plt.xticks(rotation=45)
loc = plticker.MultipleLocator(base=7.0) 
ax.xaxis.set_major_locator(loc)

sns.lineplot(
    data=df_pre_v1,
    x='yyyyww',
    y='op_value',
)


plt.xticks(rotation=45)
plt.show()

Q4. lineplot으로 가동률을 주별로 확인해 봤어요. 그래프에서 어떤 패턴을 발견할 수 있나요?

예시답안

연휴기간, 연말~연초, 휴가철 등에 가동률이 증가함을 알 수 있다.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/sajeongadongryulbunpo.max-800x600.png)

사전 가동률 데이터의 분포를 보면 위의 그림에 표시된 것 같이 연휴기간, 연말~연초, 휴가철 등에 가동률이 증가함을 알 수 있어요.

## 1-6. 상관관계 분석을 위한 준비

EDA가 끝났으니 데이터의 features들이 어떤 상관관계를 가지는지 알아봅시다.

상관관계 분석이란 두 개의 연속 변수 간의 선형 관계를 분석하는 기법으로, 한 변수가 증가하면 나머지 변수도 선형적으로 증가 또는 감소하는지를 분석해요. 두 변수 사이의 선형 관계 정도를 상관계수(correlation coefficient)라고 불러요. 상관관계 분석은 회귀 분석이나 다른 분석을 위한 사전 단계로 사용되기도 한답니다.

주의해야 할 점은 상관관계가 있다고 해서 두 변수 사이에 인과관계가 존재한다는 뜻은 아니라는 것이에요. 다시 말하면 설명 변수가 반응 변수를 예측할 때 유용할 수 있지만, 설명 변수가 반응 변수의 원인이라는 의미는 아니에요. 반응 변수가 설명 변수의 원인이 될 수도 있고, 두 변수의 관계가 인과관계보다 더 복잡할 수도 있어요.

### 기간 맞추기

___

상관관계를 보기 전에 세 데이터의 기간이 일치하는지 살펴보고, 만약 기간이 다르다면 기간을 맞추겠습니다. 우선 각 데이터의 시작 날짜와 종료 날짜를 살펴볼게요.

print(f"사전 가동률 데이터 시작 날짜: {df_pre_v1['target_dates'].unique().min()}")
print(f"존 클릭 수 데이터 시작 날짜: {df_click['target_dates'].unique().min()}")
print(f"날씨 데이터 시작 날짜: {df_weather['date'].unique().min()}")

In [ ]:
print(f"사전 가동률 데이터 종료 날짜: {df_pre_v1['target_dates'].unique().max()}")
print(f"존 클릭 수 데이터 종료 날짜: {df_click['target_dates'].unique().max()}")
print(f"날씨 데이터 종료 날짜: {df_weather['date'].unique().max()}")

존 클릭 수 데이터의 시작 날짜가 2019년 8월 31일로 가장 늦고, 날씨 데이터의 종료 날짜가 2021년 5월 22일로 가장 빠르네요. 존 클릭 수 데이터의 처음 20개의 데이터를 확인해 봅시다.

df_click.head(20)

존 클릭 수 데이터의 모든 행이 채워진 날짜가 2019년 9월 8일이군요. 따라서 모든 데이터가 있는 날짜의 시작과 끝은 아래와 같아요.

-   시작 날짜: 2019-09-08
-   종료 날짜: 2021-05-22

In [ ]:
start_date = pd.to_datetime('2019-09-08', format='%Y-%m-%d')
end_date = pd.to_datetime('2021-05-22', format='%Y-%m-%d')
len(pd.date_range(start_date, end_date))

2019년 9월 8일부터 2021년 5월 22일까지는 총 623일이고, 사전 가동률 데이터, 존 클릭 수 데이터, 날씨 데이터 각각에 시작 날짜와 종료 날짜를 적용하면 각각 623개의 데이터가 있다는 걸 알 수 있어요.

In [ ]:
print(len(df_pre[(df_pre['target_dates']>=start_date) & (df_pre['target_dates']<=end_date)]['target_dates'].unique()))
print(len(df_click[(df_click['target_dates']>=start_date) & (df_click['target_dates']<=end_date)]['target_dates'].unique()))
print(len(df_weather[(df_weather['date']>=start_date) & (df_weather['date']<=end_date)]['date'].unique()))

기간을 맞춘 각각의 데이터를 `merge()` 함수를 이용해 하나로 합쳐 줍니다.

In [ ]:
df_pre_raws = df_pre[(df_pre['target_dates']>=start_date) & (df_pre['target_dates']<=end_date)]
df_click_raws = df_click[(df_click['target_dates']>=start_date) & (df_click['target_dates']<=end_date)]
df_weather_raws = df_weather[(df_weather['date']>=start_date) & (df_weather['date']<=end_date)]

print("슝~")

In [ ]:
df_raws_semi = df_pre_raws.merge(
    df_click_raws,
    how='left',
    left_on='target_dates',
    right_on='target_dates',
    copy=False,
)

df_raws = df_raws_semi.merge(
    df_weather_raws,
    how='left',
    left_on='target_dates',
    right_on='date',
    copy=False
)

df_raws.head()

In [ ]:
df_raws.info()

`date` 컬럼은 필요 없으니까 삭제할게요. `month, year, in_clean`의 데이터타입을 `int64`로 변경하고 `week` 컬럼을 추가합시다.

In [ ]:
df_raws.drop(['date'], axis=1, inplace=True)
df_raws['month'] = df_raws['month'].astype(int)
df_raws['year'] = df_raws['year'].astype(int)
df_raws['is_clean'] = df_raws['is_clean'].astype(int)
df_raws['week'] = df_raws['target_dates'].dt.isocalendar().week

df_raws.head()

In [ ]:
df_raws.info()

훨씬 깔끔해졌나요? 마지막으로 주말과 평일의 차이를 보기 위해 `weekday`, `is_weekend` 컬럼을 추가해 볼게요.

In [ ]:
df_raws['weekday'] = df_raws['target_dates'].dt.weekday
df_raws['is_weekend'] = df_raws['weekday'] >= 5

print("슝~")

In [ ]:
# 프로젝트를 위한 데이터 저장
raws = pd.DataFrame(df_raws)
save_path = os.getenv('HOME') + '/aiffel/demand_forecasting/data/raws.csv'

raws.to_csv(save_path, index=False)

print("슝~")

In [ ]:
df_raws.head()

자, 이제 상관관계를 살펴볼 준비가 모두 끝났습니다. 본격적으로 각 features 간 상관관계를 살펴봅시다.

Q5. 가동률과 존 클릭 수, 가동률과 날씨, 가동률과 요일은 어떤 관계가 있을까요? 예상을 적어보세요.

예시답안

가동률과 존 클릭 수는 강한 양의 상관 관계가 있을 것이다. 가동률과 날씨는 별 상관이 없을 것 같다. 주말의 가동률이 더 높을 것 같다 등

## 1-7. 상관관계 분석 -- 가동률과 존 클릭수, 가동률과 날씨, 가동률과 요일

이제부터 본격적으로 상관관계 분석을 시작합시다.

#### 가동률과 존 클릭 수

___

먼저 가동률과 존 클릭 수의 상관관계를 알아봅시다. 우리가 예측할 대상인 반응 변수는 **최종 가동률('op\_rate\_0d\_all\_cars')** 이고, 설명 변수는 **존 클릭 수**에요. 이 두 변수 간의 상관관계는 어떻게 될까요?

연속형 변수에 사용되는 피어슨 상관 분석을 통해 두 변수 간 상관관계를 살펴봅시다.

In [ ]:
click_d_days = ['click_d_day', 'click_d_1', 'click_d_2', 'click_d_3', 'click_d_4', 'click_d_5', 'click_d_6', 'click_d_7']

corr_values = []

for temp_click_d_day in click_d_days:
    temp_corr_value = df_raws['op_rate_0d_all_cars'].corr(df_raws[temp_click_d_day], method='pearson')
    corr_values.append(temp_corr_value)

df_corr_op_click = pd.DataFrame([click_d_days, corr_values], index=['with_variable', 'corr']).T
df_corr_op_click

상관관계의 정도를 나타내는 상관계수(correlation coefficient)는 -1부터 1까지의 값을 가져요. 상관계수의 절대값의 크기는 직선 관계(선형 관계)에 가까운 정도이고, 부호는 직선 관계의 방향이죠. 다시 말하면 상관계수가 -1에 가까울수록 강한 음의 상관관계, 1에 가까울수록 강한 양의 상관관계라고 할 수 있어요. 반면 상관계수가 0에 가까울수록 상관관계가 매우 약함을 의미해요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seukeurinsyas_2021-09-04_23-06-24.max-800x600.png)

\[상관계수 R\]

[https://bkshin.tistory.com/entry/DATA-17-Regression](https://bkshin.tistory.com/entry/DATA-17-Regression)

표를 통해 보면 존 클릭 수와 가동률이 대부분 양의 상관관계를 가지고, d-day로 갈수록 점점 강한 상관관계를 보이고 있어요. 산점도를 통해 두 변수 간의 관계를 시각화해볼게요.

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(10, 30), sharey=True)

for i, temp_click_d_day in enumerate(click_d_days):
    row = i // 2
    col = i % 2
    sns.regplot(
        ax=axs[row, col],
        data=df_raws,
        x='op_rate_0d_all_cars',
        y=f'{temp_click_d_day}'
    )

    axs[row, col].set_title(f'{i}일 전 존 클릭 수와 당일 target_date 가동률', fontsize=10)
    axs[row, col].set_xlabel('target_date 가동률')
    axs[row, col].set_ylabel(f'{i}일 전 존 클릭 수')
    
plt.show()

N일 전 클릭 수와 당일 가동률간의 상관관계는 D-day가 가까워질수록 선형으로 설명할 수 있는 비중이 커지는 것으로 관측되고 있어요.

#### 가동률과 날씨

___

이번에는 가동률과 날씨의 상관 관계를 살펴보죠. 반응 변수는 **최종 가동률('op\_rate\_0d\_all\_cars')** 이고, 설명 변수는 **평균 기온, 평균 강수량, 강수 여부**에요. 이번에도 피어슨 상관 분석을 해볼게요.

In [ ]:
weather_variables = ['is_clean', 'avg_precipitation', 'avg_temperature']

corr_values = []

for temp_weather_variable in weather_variables:
    temp_corr_value = df_raws['op_rate_0d_all_cars'].corr(df_raws[temp_weather_variable], method='pearson')
    corr_values.append(temp_corr_value)

df_corr_weather = pd.DataFrame([weather_variables, corr_values], index=['with_variable', 'corr']).T
df_corr_weather

뚜렷한 상관관계가 잘 보이지 않네요. 하지만 **뚜렷한 상관관계가 없다고 해서 의미 없는 변수라는 뜻은 아니에요**. 특히 여러 개의 설명 변수를 가지고 있을 때는 섣불리 판단할 수 없다는 것을 명심하세요! 이번에도 산점도를 통해 시각화해봅시다.

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10, 8))

sns.scatterplot(
    ax=axs[0],
    data=df_raws,
    x='op_rate_0d_all_cars',
    y='avg_precipitation',
    hue='is_clean'
)
axs[0].set_title('날씨(평균 강우/강수)와 가동률', fontsize=15)
axs[0].set_xlabel('target_date 가동률')
axs[0].set_ylabel('평균 강우/강수')

sns.scatterplot(
    ax=axs[1],
    data=df_raws,
    x='op_rate_0d_all_cars',
    y='avg_temperature',
    hue='is_clean'
)
axs[1].set_title('날씨(평균 기온)와 가동률', fontsize=15)
axs[1].set_xlabel('target_date 가동률')
axs[1].set_ylabel('평균 기온')

plt.show()

시각화를 통해 봐도 특별한 상관관계를 살펴볼 수 없죠? 하지만 맑은 날보다 비가 오는 날에 가동률이 더 높은 것을 볼 수 있어요. 여러분들은 어떤 것을 발견했나요?

#### 가동률과 요일

___

이번에는 가동률과 요일의 상관관계를 볼게요. 반응 변수는 **최종 가동률('op\_rate\_0d\_all\_cars')** 이고, 설명 변수는 **요일('weekday')과 주말 여부 ('is\_weekend')** 에요. 요일과 주말 여부를 `groupby()`로 묶어서 요일별 평균 가동률과 평일과 주말의 평균 가동률을 알아보죠.

In [ ]:
# weekday의 값으로 0은 월요일, 1은 화요일, ..., 6은 일요일을 나타낸다.
df_raws[['op_rate_0d_all_cars', 'weekday']].groupby(['weekday']).mean()

In [ ]:
df_raws[['op_rate_0d_all_cars', 'is_weekend']].groupby(['is_weekend']).mean()

평일과 주말 중 주말의 가동률이 조금 더 높지만 평일과 아주 큰 차이는 보이지 않네요. 이번에는 바이올린 플롯을 그려서 시각화해보겠습니다.

In [ ]:
sns.violinplot(data=df_raws, x='weekday', y='op_rate_0d_all_cars')
plt.show()

In [ ]:
sns.violinplot(data=df_raws, x='is_weekend', y='op_rate_0d_all_cars')
plt.show()

시각화해보니 데이터의 분포를 더 잘 파악할 수 있죠? 일주일 중 토요일의 가동률이 가장 높고, 평일에는 비슷비슷해 보여요. 평일과 주말 중에는 주말의 가동률이 높은 것을 쉽게 파악할 수 있었어요.

이와 같이 시각화를 통해 가동률(연속형 변수)과 요일(범주형 변수)의 상관관계를 확인했지만, 상관계수를 통해서 확인해볼 수도 있습니다. 즉, ‘연속형-연속형’ 뿐만 아니라 ‘연속형-범주형’, ‘범주형-범주형’의 변수 관계를 수치화하는 상관계수도 존재합니다. 다만 다소 지엽적인 부분이므로 해당 내용이 궁금하신 분께선 ‘범주형 상관계수’의 키워드로 구글링해보시길 바래요~😉

상관관계 분석을 해보니 각 변수 간 상관관계가 잘 보였나요? 이번에는 선형 분석을 통해서 각 변수가 잘 설명되는지 볼게요.

## 1-8. 선형 회귀 분석의 이해

선형 회귀 분석은 반응 변수(또는 종속 변수)와 한 개 이상의 설명 변수(또는 독립 변수)와의 선형 상관관계를 모델링하는 회귀분석 기법이에요. 독립 변수가 1개라면 (단순) 선형 회귀, 2개 이상이라면 다중 선형 회귀라고 불러요. 선형 회귀식은 아래와 같아요.

-   단순 선형 회귀식 : $y=α_0+α_1x+ϵ$
-   다중 선형 회귀식 : $y=α_0+α_1x_1+α_2x_2+⋯+α_nx_n+ϵ$
( $x,x_n$: 설명 변수, $y$ : 반응 변수, $α_0$ : $y$ 절편, $α_n$ : 회귀계수, $ϵ$ : 오차항)

선형 회귀 그래프로 산점도를 많이 사용하는데, 산점도에서 데이터들의 경향을 알고 싶을 때 추세선을 그려요. 아래 그림의 빨간색 실선이 추세선이에요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/Screen_Shot_2022-02-04_at_2.01.43_PM.max-800x600.png)

\[산점도와 추세선\]

최적의 추세선을 그리기 위한 방법 중에 최소제곱법(Least Squares Method 또는 Ordinary Least Squares Method: OLS)이라는 것이 있어요. 데이터의 yyy값(실제값)과 추세선의 yyy값의 차를 잔차(residual)라고 하는데, 최소제곱법은 이 잔차의 제곱의 합을 최소로 하는 방법이에요.  

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seukeurinsyas_2021-09-08_16-36-01.max-800x600.png)

\[추세선과 잔차\]

[https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=9300420&logNo=222081938010](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=9300420&logNo=222081938010)

잔차의 제곱의 합은 다음 그림에 나타난 사각형들의 넓이의 합이라고 생각할 수도 있습니다. 왼쪽 그래프의 사각형들의 넓이의 합이 오른쪽 그래프의 사각형들의 넓이의 합보다 작듯이 최소제곱법을 사용하여 데이터를 가장 잘 표현하는 선형회귀선을 그릴 수 있어요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/Screen_Shot_2022-02-04_at_3.24.40_PM.max-800x600.png)

\[추세선과 잔차 제곱 시각화\]

[https://zief0002.github.io/modeling/ordinary-least-squares-ols-estimation.html](https://zief0002.github.io/modeling/ordinary-least-squares-ols-estimation.html)

### 모형과 가정 검증

___

#### 모형

`stats.model` 라이브러리를 사용해 OLS 선형 회귀 검정을 하면 아래와 같은 표가 나와요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seukeurinsyas_2021-09-08_16-58-38.max-800x600.png)

\[OLS 선형 회귀 검정\]

위의 표에서 중요한 정보는 'coef, p-value, $R^2$'에요.

coef의 여러 행 중 const는 y 절편이고, 존 클릭 수 옆에 적힌 숫자들이 회귀계수에요. 예를 들어 `click_d_1`의 경우, 선형 회귀식은 **$y=0.2145+0.1061x$** 에요. y 절편의 p-value는 0, `click_d_1`의 p-value 역시 0으로, `click_d_1`에 대한 정보는 통계적으로 유의미하다는 것을 알 수 있어요. 같은 방식으로 `click_d_2`부터 `click_d_7`까지의 선형 회귀식을 구할 수 있어요.

p-value는 '관측된 값이 통계값과 차이가 없거나 의미 있는 차이가 없다'는 귀무가설(null hypothesis)이 맞다고 전제할 때, 통계값이 실제로 관측된 값 이상일 확률이에요. 관측된 값이 귀무가설과 반대되는 정도를 0과 1 사이로 표현하고, p-value가 특정 값(0.05나 0.01 등)보다 작으면 귀무가설을 기각하죠. 일반적으로 유의수준 0.05 미만으로 나타나면 '유의미(significant)한 차이가 있다'는 뜻이랍니다.

$R^2$은 결정계수(coefficient of determination) 또는 설명력이라고 부르는데, 회귀 모형에서 설명 변수가 반응 변수를 설명하는 정도를 알려주는 지표에요. 0과 1 사이의 값을 가지며, 결정계수가 1에 가까울수록 설명 변수가 반응 변수를 많이 설명한다고 볼 수 있어요. 위의 표에서 $R^2$가 0.701이므로 설명 변수인 존 클릭 수가 반응 변수인 가동률(`op_rate_0d_all_cars`)의 약 70% 정도를 설명한다라고 할 수 있겠죠. 다만 설명 변수가 두 개 이상인 다중 선형 회귀 분석에서는 수정 결정계수(Adjusted Coefficient of determination. Adj. R-squared)를 사용합니다. 반응 변수의 변화를 설명하지 못하는 설명 변수가 모형에 추가되어도 결정계수의 값이 커질 수 있다는 사실을 고려하여, 고안된 지표입니다.

표에 대해 자세히 설명했지만 앞으로는 간단히 결정계수를 통해 설명 변수가 반응 변수인 가동률을 잘 설명하는지만 살펴볼게요.

#### 가정 검증

선형 회귀 분석에는 기본 가정 4가지가 존재해요. 잔차의 독립성, 잔차의 정규성, 잔차의 등분산성, 모형의 선형성, 이렇게 4가지이죠. 이 가정을 만족할 때 회귀 모형의 예측력이 좋다고 볼 수 있어요. 기본 가정의 각각의 의미와 검증 방법은 아래와 같아요.

1.  **잔차의 독립성**

잔차의 독립성이란 잔차가 서로 상관관계가 없이 독립적이라는 뜻이에요.

잔차가 서로 상관관계를 가지면 회귀 분석의 결과값이 아무 의미를 갖지 못하기 때문에 잔차가 서로 독립적인지 알아봐야 해요. 잔차의 독립성은 **Durbin-Watson(DW) 검정** 로 확인할 수 있어요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seukeurinsyas_2021-09-08_16-58-38_e0VXgdw.max-800x600.png)

\[Durbin-Watson\]

DW는 OLS 선형 회귀 검정의 결과로 나온 표에 나와요. 위의 그림에서 빨간색 네모 박스 안에 DW 값이 나와 있는 것이 보이시죠?

DW 값에 따라

-   1.5 ~ 2.5: 잔차는 자기 상관을 갖지 않는다. (독립적)
    
-   0.0 ~ 1.5: 잔차는 양의 자기 상관을 가진다.
    
-   2.5 ~ 4.0: 잔차는 음의 자기 상관을 가진다.
    

라고 판단할 수 있어요. 또한 0과 4에 가까워질수록 회귀 모형에 잔차가 많이 존재해 회귀 모형이 적합하지 않다고 볼 수 있습니다.

2.  **잔차의 정규성**

잔차의 정규성은 잔차가 정규분포를 따른다는 뜻입니다.

잔차의 정규성은 **Q-Q plot이나 Shapiro-Wilk 검정** 등 으로 확인할 수 있어요. 아래의 그림과 같이 **Q-Q plot에서 잔차가 대각선 방향의 직선 형태를 띠면, 즉 점들이 점선을 따라 있으면 잔차가 정규분포** 를 따른다는 뜻이에요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/original_images/seukeurinsyas_2021-09-08_16-44-11.png)

\[Q-Q plot - 점들이 점선을 따라 있으면 잔차는 정규 분포를 따릅니다.\]

Shapiro-Wilk 검정에서는 잔차가 정규분포를 따른다고 귀무가설($H_0$), 잔차가 정규분포를 따르지 않는다는 대립가설($H_1$)을 설정해요. 검정을 실행했을 때, 귀무가설이 기각되고 대립가설이 채택되면 잔차가 정규분포를 따르지 않는다는 것을 알 수 있어요.

3.  **잔차의 등분산성**

잔차의 등분산성은 회귀 모형을 통해 예측된 값은 모든 값들에 대하여 잔차의 분산이 동일하다는 의미에요.

잔차의 등분산성은 산점도를 통해 확인할 수 있어요. 그래프는 예측값(X축)에 따라 잔차가 어떻게 달라지는지 보여줘요. 정상적인 잔차는 0을 기준으로 y 값에 관계없이 특정한 패턴을 가지지 않아요. 그러나 부채꼴 모양이나 고리처럼 특정한 패턴을 보이면 잔차는 등분산성을 만족하지 않아요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/seukeurinsyas_2021-09-05_17-36-15.max-800x600.png)

\[잔차의 패턴\]

[https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=wjddudwo209&logNo=220176239447](https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=wjddudwo209&logNo=220176239447)

간단하게는 **잔차의 추세(빨간색 실선)가 수평선에 가까울수록 등분산성이 있다**고 풀이할 수 있어요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/original_images/seukeurinsyas_2021-09-08_16-44-21.png)

\[잔차의 추세(빨간색 실선)가 수평선에 가까울수록 등분산성이 있습니다.\]

4.  **모형의 선형성**

모형의 선형성은 반응 변수와 설명 변수가 선형 관계를 가진다는 가정이에요.

모형이 선형성을 가지는지 알아보려면 예측값과 잔차를 비교해야 해요. 모든 예측값에서 가운데 점선에 맞추어 잔차가 비슷하게 있어야 해요. 산점도에서 빨간색 실선은 잔차의 추세를 나타냅니다. **잔차의 추세(빨간색 실선)가 점선에서 크게 벗어난다면 예측값에 따라 잔차가 크게 달라진다** 는 것으로, 선형성이 떨어진다고 풀이할 수 있어요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/original_images/seukeurinsyas_2021-09-08_16-44-30.png)

\[잔차의 추세(빨간색 실선)가 점선에서 크게 벗어나면 선형성이 떨어집니다.\]

선형 회귀 분석의 4가지 기본 가정에 대해 더 자세한 내용이 궁금하면 아래의 링크를 참고하세요.

-   [선형 회귀분석의 4가지 기본가정](https://kkokkilkon.tistory.com/175)

## 1-9. 선형 회귀 분석 - 가동률과 존 클릭수, 가동률과 요일, 가동률과 존클릭 수 및 날씨

이제부터 본격적으로 회귀 분석 검정과 가정 검증을 해보겠습니다. **가동률과 존 클릭 수, 가동률과 날씨, 가동률과 요일, 가동률과 존 클릭 수 및 날씨(다중 회귀)** 의 순서로 진행하겠습니다. 각 회귀 분석이 선형 회귀 분석의 기본 가정 4 가지를 충족하는지에 대해 검증해보겠습니다.

### 가동률과 존 클릭 수

___

**모형**

`stats.model` 라이브러리를 사용해 최소자승법 실습을 해봅시다. 우선 설명 변수를 존 클릭 수 전체, 예측할 변수(반응 변수)를 가동률(`op_rate_0d_all_cars`)로 정해볼게요. 그리고 y 절편(상수항)을 추가해 줍니다.

In [ ]:
df_raws.sort_values(by='target_dates', inplace=True)
print("슝~")

In [ ]:
# 설명 변수 테이블 설정
_x_data = df_raws[['click_d_day', 'click_d_1', 'click_d_2', 'click_d_3', 'click_d_4', 'click_d_5', 'click_d_6', 'click_d_7']]

# 예측 대상 데이터 설정
target = df_raws['op_rate_0d_all_cars']

# 상수항 추가
x_data = sm.add_constant(_x_data, has_constant='add')

# OLS 선형회귀 검정
model = sm.OLS(target, x_data)
model_fit = model.fit()

# 결과물 출력
print(model_fit.summary())

당일 이전, 즉 적어도 1일 전에 가동률을 예측할 것이라면 d-day의 존 클릭 수는 설명 변수로 넣을 수 없어요. d-day의 존 클릭 수는 제외하고 모델링을 해볼게요.

In [ ]:
# 설명 변수 테이블 설정
_x_data = df_raws[['click_d_1', 'click_d_2', 'click_d_3', 'click_d_4', 'click_d_5', 'click_d_6', 'click_d_7']]

# 예측 대상 데이터 설정
target = df_raws['op_rate_0d_all_cars']

# 상수항 추가
x_data = sm.add_constant(_x_data, has_constant='add')

# OLS 선형회귀 검정
model = sm.OLS(target, x_data)
model_fit = model.fit()

# 결과물 출력
print(model_fit.summary())

존 클릭 수는 가동률이나 평균 기온과 달리 크기가 크기 때문에 설명 변수를 스케일링(scaling, 상수배)합시다. `stats.model`의 condition number는 스케일링에 민감한 정보이므로 유의하세요!

In [ ]:
# 설명 변수 테이블 설정
_x_data = df_raws[['click_d_1', 'click_d_2', 'click_d_3', 'click_d_4', 'click_d_5', 'click_d_6', 'click_d_7']][:]
_x_data /= 10000  

# 예측 대상 데이터 설정
target = df_raws['op_rate_0d_all_cars']

# 상수항 추가
x_data = sm.add_constant(_x_data, has_constant='add')

# OLS 선형회귀 검정
model = sm.OLS(target, x_data)
model_fit = model.fit()

# 결과물 출력
print(model_fit.summary())

위의 표에서 $R^2$가 0.701이므로 설명 변수인 존 클릭 수가 반응 변수인 가동률(`op_rate_0d_all_cars`)의 약 70% 정도를 설명한다라고 할 수 있어요.

**가정 검증**

가정을 검증해 봅니다. 설명 변수의 추정치와 잔차를 먼저 구할 거에요.

In [ ]:
# 가설 검증 재료
ols_pred = model_fit.fittedvalues  # 추정치

residual = model_fit.resid  # 잔차
print("슝~")

**1) 잔차의 독립성**

OLS 회귀 결과(OLS Regression Result)에 따르면 DW 값은 1.390이므로 잔차는 **양의 자기상관** 을 가진다고 할 수 있어요.

**2) 잔차의 정규성**

In [ ]:
sr = scipy.stats.zscore(residual)

(x, y), _ = scipy.stats.probplot(sr)

sns.scatterplot(x=x, y=y)
plt.plot(
    [-3, 3],
    [-3, 3],
    '--',
    color='grey'
)
plt.show()

Q-Q plot에서 잔차들이 대체로 점선을 따라 배치되어 있어요. 하지만 점선에서 벗어나는 잔차도 보여요. 그렇다면 Shapiro 검정의 결과는 어떻게 나오는지 살펴봅시다.

In [ ]:
# 잔차의 정규성 검정: shapiro 검정
# 귀무가설: 잔차의 정규성이 만족한다
result = scipy.stats.shapiro(residual)
print( f'# 검정 통계량: {result[0]: .5f}')
print( f'# 유의 수준:  {result[1]: .5f}')

유의 수준 5%를 가정할 경우, p = 0.0000이므로 **유의 수준 5%에서 잔차의 정규성을 만족한다는 귀무가설을 기각** 할 수 있어요. 즉 잔차가 정규성을 만족하지 않는다는 거죠.

**3) 잔차의 등분산성**

In [ ]:
sns.regplot(
    x=ols_pred,
    y=np.sqrt(np.abs(sr)),
    lowess=True,
    line_kws={'color':'red'}    
)
plt.show()

잔차의 추세(빨간색 실선)가 수평선에서 조금 벗어난 것 같아요.

**4) 모형의 선형성**

In [ ]:
잔차의 추세(빨간색 실선)가 수평선에서 조금 벗어난 것 같아요.

**4) 모형의 선형성**

잔차의 추세(빨간색 실선)가 점선과 비슷한 부분도 있지만 점선에서 벗어나는 부분도 있죠? 이럴 때는 어떻게 판단하면 좋을까요?

### 가동률과 날씨

___

**모형**

In [ ]:
# 설명 변수 테이블 설정
_x_data = df_raws[['is_clean', 'avg_precipitation', 'avg_temperature']]

# 예측 대상 데이터 설정
target = df_raws['op_rate_0d_all_cars']

# 상수항 추가
x_data = sm.add_constant(_x_data, has_constant='add')

# OLS 선형회귀 검정
model = sm.OLS(target, x_data)
model_fit = model.fit()

# 결과물 출력
print(model_fit.summary())

회귀직선의 $R^2$값이 굉장히 낮네요. 기온이 너무 높거나 낮을 때 가동률이 낮아질 수 있으므로 기온의 2차항까지 회귀식에 포함시켜 볼게요. 회귀식은 보통 1차식(직선)이지만 2차항을 넣어주면 곡선관계도 분석할 수 있어요.

In [ ]:
# 제곱 항 생성
df_raws['avg_temperature_sq'] = df_raws['avg_temperature'] ** 2

# 설명 변수 테이블 설정
_x_data = df_raws[['is_clean', 'avg_precipitation', 'avg_temperature', 'avg_temperature_sq']]

# 예측 대상 데이터 설정
target = df_raws['op_rate_0d_all_cars']

# 상수항 추가
x_data = sm.add_constant(_x_data, has_constant='add')

# OLS 선형회귀 검정
model = sm.OLS(target, x_data)
model_fit = model.fit()

# 결과물 출력
print(model_fit.summary())

2차항을 넣어주어도 $R^2$이 여전히 낮기 때문에 날씨가 가동률을 예측하는데에 큰 도움이 되지 않음을 확인할 수 있어요.

**가정 검증**

In [ ]:
# 가설 검증 재료
ols_pred = model_fit.fittedvalues

residual = model_fit.resid
print("슝~")

**1) 잔차의 독립성**

OLS 회귀 결과에 따르면 DW 값은 0.993이므로 잔차는 **양의 자기 상관** 을 가진다고 판단할 수 있어요.

**2) 잔차의 정규성**

In [ ]:
sr = scipy.stats.zscore(residual)

(x, y), _ = scipy.stats.probplot(sr)

sns.scatterplot(x=x, y=y)
plt.plot(
    [-3, 3],
    [-3, 3],
    '--',
    color='grey'
)
plt.show()

In [ ]:
# 잔차의 정규성 검정: shapiro 검정
# 귀무가설: 잔차의 정규성이 만족한다
result = scipy.stats.shapiro(residual)
print( f'# 검정 통계량: {result[0]: .5f}')
print( f'# 유의 수준:  {result[1]: .5f}')

유의 수준 5%를 가정할 경우, p = 0.0000이므로 **유의 수준 5%에서 잔차의 정규성을 만족한다는 귀무가설을 기각** 할 수 있어요. Q-Q plot에서도 잔차들이 점선에서 벗어나 있죠.

**3) 잔차의 등분산성**

In [ ]:
sns.regplot(
    x=ols_pred,
    y=np.sqrt(np.abs(sr)),
    lowess=True,
    line_kws={'color':'red'}    
)
plt.show()

잔차의 추세(빨간색 실선)가 수평선에서 조금 벗어나 있네요.

**4) 모형의 선형성**

In [ ]:
sns.regplot(
    x=ols_pred,
    y=residual,
    lowess=True,
    line_kws={'color':'red'}
)

plt.plot(
    [ols_pred.min(), ols_pred.max()],
    [0, 0],
    '--',
    color='grey'
)
plt.show()

잔차의 추세(빨간색 실선)가 점선과 거의 비슷하죠? 가동률과 날씨 모형은 선형성을 띠고 있어요.

### 가동률과 요일

___

**모형**

In [ ]:
# 설명 변수 테이블 설정
_x_data = df_raws[['weekday']][:]

# Dummy variable 생성
_x_data = pd.get_dummies(_x_data['weekday'], prefix='weekday', drop_first=False)

# 예측 대상 데이터 설정
target = df_raws['op_rate_0d_all_cars']

# 상수항 추가
x_data = sm.add_constant(_x_data, has_constant='add')

# OLS 선형회귀 검정
model = sm.OLS(target, x_data)
model_fit = model.fit()

# 결과물 출력
print(model_fit.summary())

$R^2$가 0.436이므로 요일이 가동률을 약 40% 정도 설명한다고 볼 수 있어요.

**가설검증**

In [ ]:
# 가설 검증 재료
ols_pred = model_fit.fittedvalues

residual = model_fit.resid
print("슝~")

**1) 잔차의 독립성**

OLS 회귀 결과에 따르면 DW 값은 0.605이므로 잔차는 **양의 자기 상관**을 가진다고 판단할 수 있어요.

**2) 잔차의 정규성**

In [ ]:
sr = scipy.stats.zscore(residual)

(x, y), _ = scipy.stats.probplot(sr)

sns.scatterplot(x=x, y=y)
plt.plot(
    [-3, 3],
    [-3, 3],
    '--',
    color='grey'
)
plt.show()

In [ ]:
# 잔차의 정규성 검정: shapiro 검정
# 귀무가설: 잔차의 정규성이 만족한다
result = scipy.stats.shapiro(residual)
print( f'# 검정 통계량: {result[0]: .5f}')
print( f'# 유의 수준:  {result[1]: .5f}')

유의 수준 5%를 가정할 경우, p = 0.0000이므로 **유의 수준 5%에서 잔차의 정규성을 만족한다는 귀무가설을 기각** 할 수 있어요. Q-Q plot에서도 같은 결과를 보여주고 있군요.

**3) 잔차의 등분산성**

In [ ]:
sns.regplot(
    x=ols_pred,
    y=np.sqrt(np.abs(sr)),
    lowess=True,
    line_kws={'color':'red'}    
)
plt.show()

**4) 모형의 선형성**

In [ ]:
sns.regplot(
    x=ols_pred,
    y=residual,
    lowess=True,
    line_kws={'color':'red'}
)

plt.plot(
    [ols_pred.min(), ols_pred.max()],
    [0, 0],
    '--',
    color='grey'
)
plt.show()

### 가동률과 존 클릭 수 및 날씨(다중 회귀)

___

이번에는 다중회귀를 해보려고 해요. 다중 회귀라고 해서 특별할 건 없어요. 선형 회귀는 설명 변수가 1개였지만 다중 회귀는 설명 변수가 2개 이상이라는 것 외에는 별 차이가 없답니다.

**모형**

설명 변수로 존 클릭 수, 날씨, 그리고 요일을 넣어줍시다.

In [ ]:
# 설명 변수 테이블 설정
_x_data = df_raws[['click_d_1', 'click_d_2', 'click_d_3', 'click_d_4', 'click_d_5', 'click_d_6', 'click_d_7', 'is_clean', 'avg_precipitation', 'avg_temperature']]

_x_weekday = pd.get_dummies(df_raws['weekday'], prefix='weekday', drop_first=False)

_x_data = pd.concat([_x_data, _x_weekday], axis=1)

# 예측 대상 데이터 설정
target = df_raws['op_rate_0d_all_cars']

# 상수항 추가
x_data = sm.add_constant(_x_data, has_constant='add')

# OLS 선형회귀 검정
model = sm.OLS(target, x_data)
model_fit = model.fit()

# 결과물 출력
print(model_fit.summary())

$R^2$값이 단순 선형 회귀 분석을 할 때보다 높으므로 가동률을 더 잘 예측한다고 볼 수 있어요.

**가정 검증**

In [ ]:
# 가설 검증 재료
ols_pred = model_fit.fittedvalues

residual = model_fit.resid
print("슝~")

**1) 잔차의 독립성**

OLS 회귀 결과에 따르면 DW 값은 1.050이므로 잔차는 **양의 자기 상관** 을 가진다고 판단할 수 있어요.

**2) 잔차의 정규성**

In [ ]:
sr = scipy.stats.zscore(residual)

(x, y), _ = scipy.stats.probplot(sr)

sns.scatterplot(x=x, y=y)
plt.plot(
    [-3, 3],
    [-3, 3],
    '--',
    color='grey'
)
plt.show()

In [ ]:
# 잔차의 정규성 검정: shapiro 검정
# 귀무가설: 잔차의 정규성이 만족한다
result = scipy.stats.shapiro(residual)
print( f'# 검정 통계량: {result[0]: .5f}')
print( f'# 유의 수준:  {result[1]: .5f}')

유의 수준 5%를 가정할 경우, p = 0.0000이므로 **유의 수준 5%에서 잔차의 정규성을 만족한다는 귀무가설을 기각** 할 수 있어요.

**3) 잔차의 등분산성**

In [ ]:
sns.regplot(
    x=ols_pred,
    y=np.sqrt(np.abs(sr)),
    lowess=True,
    line_kws={'color':'red'}    
)
plt.show()

4) 모형의 선형성

In [ ]:
sns.regplot(
    x=ols_pred,
    y=residual,
    lowess=True,
    line_kws={'color':'red'}
)

plt.plot(
    [ols_pred.min(), ols_pred.max()],
    [0, 0],
    '--',
    color='grey'
)
plt.show()

지금까지 OLS 선형 회귀 검정과 가정 검증을 해봤어요.

정리하면, 다중 회귀 검정(가동률과 존 클릭 수 및 날씨)이 가동률을 가장 예측을 잘 했어요. 하지만 가정 검증에서 가정을 만족하지 않는 경우가 있었어요. 즉 4가지 가정을 만족하지 못하므로 선형 회귀 모형이 잘 예측한다고 할 수 없다는 거에요. 이런 경우 WLS(Weighted Least Squares), GLS(Generalized Least Squares) 등의 방법을 적용할 수 있지만, 프로젝트 노드에서 더 예측을 잘 할 수 있는 다른 모형을 사용하고자 합니다.

## 1-10. 마무리하며

지금까지 변수들 간의 관계를 확인하기 위해 상관관계 분석과 선형 회귀 분석을 진행해봤어요.

![content img](https://d3s0tskafalll9.cloudfront.net/media/images/Business_coach_showing_growth_graph_to_busin.max-800x600.jpg)

먼저 결측치를 처리하고 데이터 타입을 맞춤으로써 데이터의 일관성을 확인하고, EDA를 통해 데이터를 기본 분포를 살펴보았죠. 존 클릭 수 데이터로부터는 회원의 대부분이 이용 당일에 차량을 예약하는 패턴을 있음을, 사전 가동률 분포 데이터에서는 연휴기간, 연말~연초, 휴가철 등에 가동률이 증가함을 알 수 있었어요.

상관관계 분석에서는 가동률과 존 클릭 수, 가동률과 날씨, 가동률과 요일의 관계가 어떻게 나타나는지 보았어요. D-day로 갈수록 가동률과 존 클릭수는 점점 강한 양의 상관관계를 보였어요. 반면 가동률과 날씨는 상관관계가 잘 보이지 않지만 비가 오면 가동률이 더 높아지고, 매우 춥거나 더우면 가동률이 높지 않은 것을 알 수 있었죠. 가동률과 요일에서는 평일과 주말의 차이가 크게 보이지 않았지만, 주말이 가동률과 조금 더 높은 양의 상관관계를 보인다는 것을 알 수 있었어요.

선형 회귀 분석에서는 가동률과 존 클릭 수, 가동률과 날씨, 가동률과 요일을 단순 선형 회귀 분석하였고, 가동률과 존 클릭 수 및 날씨를 다룬 다중 회귀 분석까지 해 보았어요. 각각의 경우 모형을 살펴본 후, 가정 검증을 했었죠. 단순 선형 회귀 분석에서는 날씨나 요일보다 존 클릭 수가 가동률을 잘 설명했었어요. 그러나 1개의 변수보다 여러 개의 변수를 고려했을 때, 가동률이 가장 잘 예측되었어요.

하지만 상관관계 분석과 선형 회귀 분석의 결과로 차량의 수요를 완벽히 예측할 수는 없어요. 다음 시간에는 시계열 특성을 고려한 모형을 사용해 차량 수요 예측을 해 볼 거에요. 어떤 모형을 사용할 수 있을지 생각해보세요.